# Fuzzy Control Systems: El Problema de la Propina

**Disciplina:** Lógica Difusa, Sistemas de Control

**Objetivo:**
El objetivo de este notebook es ilustrar los principios de la lógica difusa para generar un comportamiento complejo a partir de un conjunto compacto e intuitivo de reglas expertas, utilizando el "problema de la propina" como ejemplo. Se creará un sistema de control difuso para modelar cómo se podría decidir la propina en un restaurante basándose en la calidad del servicio y la comida.

## 1. Carga de Librerías

Para este ejercicio, utilizaremos `numpy` para la creación de rangos numéricos y `skfuzzy` para las funcionalidades de lógica difusa y control. Específicamente, `skfuzzy.control` nos proporciona la API para construir sistemas de control difuso.

In [14]:
# Configuración para el entorno de trabajo
%reload_ext autoreload
%autoreload 2

# Configuración para mostrar gráficos inline en Jupyter Notebook (opcional en script, pero útil para notebook)
# Esto se activaría en una celda de código de notebook
%matplotlib inline

In [15]:
# Importar librerías necesarias
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt # Importar matplotlib para visualizaciones si es necesario fuera de .view()
import matplotlib

matplotlib.use('TkAgg')

## 2. Definición del Problema de la Propina

Crearemos un sistema de control difuso que modela cómo podrías decidir dar propina en un restaurante. Al dar propina, consideras la calidad del servicio y la calidad de la comida, calificadas entre 0 y 10. Usas esto para dejar una propina de entre 0 y 25%.

Formularíamos este problema como:

* **Antecedentes (Entradas)**
   - `calidad_comida` (quality)
      * Universo (rango de valores nítidos): ¿Qué tan sabrosa fue la comida, en una escala de 0 a 10?
      * Conjuntos difusos (rango de valores difusos): mala, decente, excelente (poor, acceptable, amazing en el original, se adaptará a 3 niveles)
   - `servicio` (service)
      * Universo: ¿Qué tan bueno fue el servicio del personal, en una escala de 0 a 10?
      * Conjuntos difusos: pobre, aceptable, increíble (poor, acceptable, amazing en el original, se adaptará a 3 niveles)
* **Consecuentes (Salidas)**
   - `propina` (tip)
      * Universo: ¿Cuánta propina deberíamos dejar, en una escala de 0% a 25%?
      * Conjuntos difusos: baja, media, alta
* **Reglas**
   - SI el *servicio* fue bueno O la *calidad de la comida* fue buena, ENTONCES la propina será alta.
   - SI el *servicio* fue aceptable, ENTONCES la propina será media.
   - SI el *servicio* fue pobre Y la *calidad de la comida* fue pobre, ENTONCES la propina será baja.
* **Uso**
   - Si le digo a este controlador que califiqué:
      * el servicio como 9.8, y
      * la calidad de la comida como 6.5,
   - recomendaría dejar:
      * una propina del (aproximadamente) 20.2%.

## 3. Desarrollo del Sistema de Control Difuso

### 3.1. Definición de Variables del Universo (Antecedentes y Consecuentes)

**Objetivo:** Crear los objetos Antecedente y Consecuente que contienen las variables del universo y sus funciones de pertenencia.

In [16]:
# Los nuevos objetos Antecedent/Consequent almacenan las variables del universo y las funciones de pertenencia.
calidad_comida = ctrl.Antecedent(np.arange(0, 11, 1), 'calidad_comida') # Renombrado de 'quality'
servicio = ctrl.Antecedent(np.arange(0, 11, 1), 'servicio')
propina = ctrl.Consequent(np.arange(0, 26, 1), 'propina') # Renombrado de 'tip'

### 3.2. Funciones de Pertenencia Automáticas y Personalizadas

**Objetivo:** Poblar las funciones de pertenencia. Usaremos funciones automáticas para las entradas y personalizadas para la salida.

**Características:**
* `automf(n)`: Puede generar automáticamente `n` funciones de pertenencia (típicamente 3, 5 o 7). Para este ejemplo, usaremos 3 para `calidad_comida` y `servicio`, que `skfuzzy` nombrará como 'poor', 'average', 'good' (o similar, dependiendo de la versión y la lógica interna, aunque el script original usa poor, acceptable, amazing; automf(3) generará nombres genéricos que podemos remapear conceptualmente o usar directamente).
* Funciones personalizadas: Se construirán interactivamente para la variable `propina` usando funciones triangulares (`trimf`).

In [17]:
# La población automática de funciones de pertenencia es posible con .automf(3, 5 o 7)
# Esto generará funciones llamadas 'poor', 'average', 'good' para cada una.
calidad_comida.automf(3, names=['pobre', 'decente', 'excelente']) # Usando names para etiquetas en español
servicio.automf(3, names=['pobre', 'aceptable', 'increible'])    # Usando names para etiquetas en español


# Las funciones de pertenencia personalizadas se pueden construir interactivamente con una API familiar,
# similar a Python.
propina['baja'] = fuzz.trimf(propina.universe, [0, 0, 13])
propina['media'] = fuzz.trimf(propina.universe, [0, 13, 25])
propina['alta'] = fuzz.trimf(propina.universe, [13, 25, 25])

### 3.3. Visualización de las Funciones de Pertenencia

**Objetivo:** Entender cómo se ven las funciones de pertenencia definidas.

**Método:** Utilizaremos el método `.view()` para cada variable difusa. En un entorno Jupyter, los gráficos se muestran inline. En otros entornos, podría ser necesario `plt.show()`.

In [18]:
# Puedes ver cómo se ven estas funciones con .view()
# calidad_comida['decente'].view() # El script original visualizaba una MF específica 'average'
# Aquí visualizaremos todas las funciones de pertenencia para cada variable.
calidad_comida.view()
plt.title('Funciones de Pertenencia para Calidad de la Comida') # Añadir título para claridad
# Se añade plt.show() para asegurar la visualización en scripts, aunque en notebooks .view() suele ser suficiente.
# Para formato Percent Script, es mejor asumir que se necesita plt.show() si no es la última línea de la celda.
# No obstante, .view() a menudo maneja esto internamente en algunos backends.
# Si no se muestra en el notebook, descomentar plt.show() o ejecutar en celdas separadas.

Text(0.5, 1.0, 'Funciones de Pertenencia para Calidad de la Comida')

In [19]:
servicio.view()
plt.title('Funciones de Pertenencia para Servicio')

Text(0.5, 1.0, 'Funciones de Pertenencia para Servicio')

In [20]:
propina.view()
plt.title('Funciones de Pertenencia para Propina')

Text(0.5, 1.0, 'Funciones de Pertenencia para Propina')

### 3.4. Definición de las Reglas Difusas

**Objetivo:** Definir la relación difusa entre las variables de entrada y salida.

**Reglas a Implementar:**
1. SI la *calidad de la comida* es pobre (pobre) O el *servicio* es pobre (pobre), ENTONCES la propina será baja.
2. SI el *servicio* es aceptable (aceptable), ENTONCES la propina será media.
3. SI la *calidad de la comida* es buena (excelente) O el *servicio* es bueno (increible), ENTONCES la propina será alta.

**Nota sobre etiquetas:** `automf(3)` en `skfuzzy` por defecto crea etiquetas como 'poor', 'average', 'good'. Usaremos estas etiquetas generadas o las personalizadas si se definieron con `names`.
Script original usa: `quality['poor']`, `service['poor']`, `service['average']`, `service['good']`, `quality['good']`.
Adaptaremos a las etiquetas generadas/personalizadas:
calidad_comida: 'pobre', 'decente', 'excelente'
servicio: 'pobre', 'aceptable', 'increible'

In [21]:
# Definición de las reglas
# Usar las etiquetas que automf(3, names=[...]) creó o las que se definieron manualmente.
rule1 = ctrl.Rule(calidad_comida['pobre'] | servicio['pobre'], propina['baja'])
rule2 = ctrl.Rule(servicio['aceptable'], propina['media'])
rule3 = ctrl.Rule(servicio['increible'] | calidad_comida['excelente'], propina['alta'])

### 3.5. Creación y Simulación del Sistema de Control

**Objetivo:** Crear el sistema de control a partir de las reglas y simularlo con entradas específicas.

**Proceso:**
1. Crear un objeto `ControlSystem` con la lista de reglas.
2. Crear un objeto `ControlSystemSimulation` para aplicar el controlador a circunstancias específicas.
3. Especificar los valores de entrada para `calidad_comida` y `servicio`.
4. Ejecutar el método `compute()` para obtener la salida.

In [22]:
# Creación del sistema de control
sistema_ctrl_propina = ctrl.ControlSystem([rule1, rule2, rule3]) # Renombrado de tipping_ctrl

**Simulación:**
Para simular este sistema de control, crearemos un `ControlSystemSimulation`. Este objeto representa nuestro controlador aplicado a un conjunto específico de circunstancias.

In [23]:
# Creación de la simulación del sistema de control
simulacion_propina = ctrl.ControlSystemSimulation(sistema_ctrl_propina) # Renombrado de tipping

Ahora podemos simular nuestro sistema de control especificando las entradas y llamando al método `compute`. Supongamos que calificamos la calidad de la comida como 6.5 de 10 y el servicio como 9.8 de 10.

In [24]:
# Pasar entradas al ControlSystem usando las etiquetas de los Antecedentes
simulacion_propina.input['calidad_comida'] = 6.5
simulacion_propina.input['servicio'] = 9.8

# Realizar los cálculos
simulacion_propina.compute()

### 3.6. Visualización del Resultado

**Objetivo:** Mostrar el valor de salida calculado y visualizarlo.

In [25]:
# Obtener y mostrar la salida
valor_propina = simulacion_propina.output['propina']
print(f"El valor de la propina sugerida es: {valor_propina:.2f}%")

El valor de la propina sugerida es: 19.85%


In [26]:
# Visualizar la variable de salida con el resultado de la simulación
propina.view(sim=simulacion_propina)
plt.title('Salida de la Propina con Agregación y Desfusificación')

Text(0.5, 1.0, 'Salida de la Propina con Agregación y Desfusificación')

## 4. Conclusiones del Ejercicio

**Interpretación del Resultado:**
El valor de la propina resultante (por ejemplo, 19.84% en el script original) es la recomendación del sistema de control difuso basado en las entradas y las reglas definidas.

**Reflexiones Finales:**
El poder de los sistemas difusos radica en permitir un comportamiento complicado e intuitivo basado en un sistema disperso de reglas con una sobrecarga mínima. Es importante notar que, aunque los universos de nuestras funciones de pertenencia eran relativamente simples (definidos en enteros), `fuzz.interp_membership` permite que la resolución efectiva aumente bajo demanda. Este sistema puede responder a cambios arbitrariamente pequeños en las entradas, y la carga de procesamiento es mínima.

Este ejemplo demuestra cómo se puede modelar un problema de toma de decisiones del mundo real utilizando lógica difusa, transformando reglas lingüísticas y conocimiento experto en un sistema automatizado.